In [1]:
!pip install duckdb --no-index --find-links=file:///kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [ ]:
!pip install polars[numpy,pandas,pyarrow] --no-index --find-links=file:///kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/polars_pkg

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import duckdb as dd
import polars as pl
import pyarrow
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
!pip install plotly
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
!pip install -U imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [4]:
# Path to the directory containing your audio dataset
dataset_dir = '/kaggle/input/birdclef-2024/train_audio'
# Initialize an empty dictionary to store the mapping between audio files and labels
label_mapping = {}
# Iterate over subdirectories (classes) in the dataset directory
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    # Check if the item in the dataset directory is a directory
    if os.path.isdir(label_dir):
        # Iterate over audio files in the subdirectory (class)
        for audio_file in os.listdir(label_dir):
            # Add the mapping between audio file path and label to the dictionary
            audio_file_path = os.path.join(label_dir, audio_file)
            label_mapping[audio_file_path] = label
            
# label_mapping

# Create a list of tuples containing the audio file paths and labels
data = [(audio_file_path, label) for audio_file_path, label in label_mapping.items()]
# Create a Pandas DataFrame from the list of tuples
annotated_data = pd.DataFrame(data, columns=['audio_file_path', 'label'])

label_encoder = LabelEncoder()
annotated_data['encoded_label'] = label_encoder.fit_transform(annotated_data['label'])

annotated_data.head(5)

,audio_file_path,label,encoded_label
0,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,137
1,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,137
2,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,137
3,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,137
4,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,137


In [5]:
annotated_data.shape

(24459, 3)

In [ ]:
def get_file_duration(full_file_path):
    duration = librosa.get_duration(path=full_file_path, sr=32000)
    return round(duration,2)

vect_func = np.vectorize(get_file_duration)

annotated_data['file_duration'] = vect_func(annotated_data['audio_file_path'])

In [ ]:
dur_stats_per_species = dd.sql("select label, min(file_duration) as min_duration, max(file_duration) as max_duration \
, avg(file_duration) as avg_duration from annotated_data group by label").pl().sort(by='avg_duration')

dur_stats_per_species_melted = dur_stats_per_species.melt(id_vars="label", value_vars=["min_duration","max_duration","avg_duration"])

In [ ]:
fig0 = px.bar(dur_stats_per_species, x='avg_duration', y='label', orientation='h')
fig0.show()

In [ ]:
fig_hist_files = px.histogram(dur_stats_per_species_melted, x="value", color="variable")
fig_hist_files.show()

In [ ]:
fig0 = px.bar(dur_stats_per_species.sort(by='min_duration'), x='min_duration', y='label', orientation='h')
fig0.show()

In [ ]:
fig0 = px.bar(dur_stats_per_species.sort(by='max_duration'), x='max_duration', y='label', orientation='h')
fig0.show()

In [ ]:
files_per_species = dd.sql("select label, count(distinct(audio_file_path)) as files from annotated_data group by label")\
.pl().sort(by='files')

In [ ]:
fig = px.bar(files_per_species, x='files', y='label', orientation='h')
fig.show()

In [ ]:
fig_hist = px.histogram(files_per_species, x="files")
fig_hist.show()

In [ ]:
file_w_max_dur = dd.sql("select ad.*, replace(ad.audio_file_path, '/kaggle/input/birdclef-2024/train_audio/', '') as file_name_w_label from annotated_data ad \
join (select label, max(file_duration) as max_duration from annotated_data group by label) agg \
on ad.label = agg.label and ad.file_duration = agg.max_duration order by ad.label\
").pl()

max_duration_files = file_w_max_dur.group_by("file_name_w_label").agg(pl.col("file_duration").max()).sort(by='file_duration')

In [ ]:
"""pd.set_option(
    'display.max_colwidth', 1000
)"""
pl.Config.set_fmt_str_lengths(100)
max_duration_files.tail(10)

In [ ]:
fig2 = px.bar(max_duration_files.tail(50), x='file_duration', y='file_name_w_label', orientation='h')
fig2.show()

In [ ]:
def audio_waveframe(file_path):
    # Load the audio file
    audio_data, sampling_rate = librosa.load(file_path)
    # Calculate the duration of the audio file
    duration = len(audio_data) / sampling_rate
    # Create a time array for plotting
    time = np.arange(0, duration, 1/sampling_rate)
    # Plot the waveform
    plt.figure(figsize=(30, 4))
    plt.plot(time, audio_data, color='blue')
    plt.title('Audio Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plot = plt.show()
    return plot

def spectrogram(file_path):
    # Compute the short-time Fourier transform (STFT)
    n_fft = 500  # Number of FFT points 2048
    hop_length = 50  # Hop length for STFT 512
    audio_data, sampling_rate = librosa.load(file_path)
    stft = librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length)
    # Convert the magnitude spectrogram to decibels (log scale)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    # Plot the spectrogram
    plt.figure(figsize=(30, 6))
    librosa.display.specshow(spectrogram, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='linear')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plot = plt.show()
    return plot

def audio_analysis(file_path):
    aw = audio_waveframe(file_path)
    spg = spectrogram(file_path)
    return aw, spg

In [ ]:
audio_analysis(max_audio_file_path)
Audio(max_audio_file_path)

In [ ]:
meta_data = pl.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv', low_memory=True)
meta_data.head(5)

In [ ]:
meta_data.describe()

In [ ]:
fig = px.scatter_mapbox(meta_data, lat='latitude', lon='longitude', color='common_name', 
                        hover_name='common_name', hover_data=['latitude', 'longitude'], 
                        title='Origin of Bird Species',
                        zoom=1, height=600, template='plotly_dark')
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.show()

In [ ]:
meta_data.filter((pl.col('primary_label')=='redspu1') & (pl.col('filename')=='redspu1/XC325427.ogg'))\
.select('type','latitude','longitude','common_name','rating')

#### Possibility of a feature with the type of recording, latitude & longitude
#### the rating can be used as a weight

In [ ]:
meta_data.filter((pl.col('primary_label')=='redspu1') & (pl.col('filename')=='redspu1/XC325427.ogg')).select('type')

In [ ]:
audio_data, sampling_rate = librosa.load('/kaggle/input/birdclef-2024/train_audio/redspu1/XC325427.ogg')
print(sampling_rate)

In [ ]:
duration = librosa.get_duration(y=audio_data, sr=sampling_rate)
duration

In [ ]:
print(annotated_data['audio_file_path'].iloc[10])
print(annotated_data['label'].iloc[10])

In [ ]:
annotated_data.shape[0]

In [ ]:
# Function to extract features from audio file
"""def extract_features(file_path):
    # Load audio file
    audio, sample_rate = librosa.load(file_path)
    # Extract features using Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Flatten the features into a 1D array
    flattened_features = np.mean(mfccs.T, axis=0)
    return flattened_features"""

In [ ]:
"""def extract_features_five_sec(file_path):
    audio, sample_rate = librosa.load(file_path, sr=32000)
    samples_per_segment = sample_rate * 5
    if len(audio) != 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)
        
    for i in range(0, total_samples+160000, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            chunk = cur_file+'_'+str(int((i/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0))"""

In [5]:
"""def load_data_and_extract_features_five_sec():
    labels = []
    features = []
    # Loop through each audio file in the dataset directory
    for i in range(annotated_data.shape[0]):
        # labels.append(annotated_data['label'].iloc[i])
        file_path = annotated_data['audio_file_path'].iloc[i]
        audio, sample_rate = librosa.load(file_path, sr=32000)
        samples_per_segment = sample_rate * 5
        if len(audio) > 7680000:
            total_samples = 7680000
        else:
            total_samples = len(audio)
            
        for j in range(0, total_samples+160000, samples_per_segment):
            if j + samples_per_segment <= total_samples:
                segment = audio[j:j + samples_per_segment]
                # chunk = cur_file+'_'+str(int((j/32000)+5))
                mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
                flattened_features = (np.mean(mfccs.T, axis=0))
                features.append(flattened_features)
                labels.append(annotated_data['label'].iloc[i])
        
        # features.append(extract_features(annotated_data['audio_file_path'].iloc[i]))
    
    return np.array(features), np.array(labels)"""

In [ ]:
"""def load_data_and_extract_features():
    labels = []
    features = []
    # Loop through each audio file in the dataset directory
    for i in range(annotated_data.shape[0]):
        labels.append(annotated_data['label'].iloc[i])
        features.append(extract_features(annotated_data['audio_file_path'].iloc[i]))
    
    return np.array(features), np.array(labels)"""

In [8]:
"""from tqdm import tqdm
# extracted_training_features, labels = load_data_and_extract_features()

extracted_training_features_five_sec, labels_five_sec = load_data_and_extract_features_five_sec()

# for i in tqdm(annotated_data['audio_file_path']):
#     features = extract_features(file_path=i)   
#     # print(features)
#     extracted_features.append(features)"""

In [4]:
from tqdm import tqdm

labels = []
features = []

# Loop through each audio file in the dataset directory
for i in tqdm(range(annotated_data.shape[0]), desc = 'Outer Loop'):
    # labels.append(annotated_data['label'].iloc[i])
    file_path = annotated_data['audio_file_path'].iloc[i]
    audio, sample_rate = librosa.load(file_path, sr=32000)
    samples_per_segment = sample_rate * 5
    if len(audio) > 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)

    for j in range(0, total_samples+160000, samples_per_segment):
        if j + samples_per_segment <= total_samples:
            segment = audio[j:j + samples_per_segment]
            # chunk = cur_file+'_'+str(int((j/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0))
            features.append(flattened_features)
            labels.append(annotated_data['encoded_label'].iloc[i])

Outer Loop: 100%|██████████| 24459/24459 [1:58:10<00:00,  3.45it/s]  


In [5]:
extracted_training_features_five_sec = np.array(features)
labels_five_sec = np.array(labels)

In [6]:
with open("extracted_training_features_five_sec", "wb") as file:   #Pickling
    pickle.dump(extracted_training_features_five_sec, file)

In [7]:
with open("labels_five_sec", "wb") as file:   #Pickling
    pickle.dump(labels_five_sec, file)

In [7]:
with open("/kaggle/input/features-and-labels-five-sec-v2/extracted_training_features_five_sec", "rb") as file:
    pickled_extracted_features_five_sec = pickle.load(file)
    
with open("/kaggle/input/features-and-labels-five-sec-v2/labels_five_sec", "rb") as file:
    labels_five_sec = pickle.load(file)

In [85]:
with open('/kaggle/input/extracted-features-pickle/extracted_features', "rb") as file:
    pickled_extracted_features = pickle.load(file)
    
x = np.vstack(pickled_extracted_features)
y = annotated_data['encoded_label']

print(x.shape)
print(y.shape)

(24459, 40)
(24459,)


In [6]:
class_list = annotated_data['encoded_label'].unique().tolist()
len(class_list)

182

In [34]:
"""from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
"""
x_five_sec = np.vstack(pickled_extracted_features_five_sec)
#np.vstack(pickled_extracted_features)
y_five_sec = labels_five_sec
#annotated_data['encoded_label']
#y = label_binarize(annotated_data['encoded_label'], classes=class_list)

print(x_five_sec.shape)
print(y_five_sec.shape)
#print(y_multiclass.shape)

(169774, 40)
(169774,)


In [9]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

In [35]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#print("full train shape & label shape = {0} & {1}".format(x_train.shape, y_train.shape))
#print("full test shape & label shape = {0} & {1}".format(x_test.shape, y_test.shape))

"""x_five_sec = x[:24459]
y_five_sec = y[:24459]"""

skf = StratifiedKFold(n_splits=5)
# skf.get_n_splits(x, y)
for i, (train_index, test_index) in enumerate(skf.split(x_five_sec, y_five_sec)):
    print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    print("train feature shape & label shape = {0} & {1}".format(x_five_sec[train_index].shape, y_five_sec[train_index].shape))
    #print(f"  Test:  index={test_index}")
    print("test feature shape & label shape = {0} & {1}".format(x_five_sec[test_index].shape,y_five_sec[test_index].shape))

Fold 0:
train feature shape & label shape = (135819, 40) & (135819,)
test feature shape & label shape = (33955, 40) & (33955,)
Fold 1:
train feature shape & label shape = (135819, 40) & (135819,)
test feature shape & label shape = (33955, 40) & (33955,)
Fold 2:
train feature shape & label shape = (135819, 40) & (135819,)
test feature shape & label shape = (33955, 40) & (33955,)
Fold 3:
train feature shape & label shape = (135819, 40) & (135819,)
test feature shape & label shape = (33955, 40) & (33955,)
Fold 4:
train feature shape & label shape = (135820, 40) & (135820,)
test feature shape & label shape = (33954, 40) & (33954,)


In [36]:
x_train, x_test, y_train, y_test = train_test_split(x_five_sec[train_index], y_five_sec[train_index], test_size=0.45, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(74701, 40)
(74701,)
(61119, 40)
(61119,)


In [37]:
print("train feature shape & label shape = {0} & {1}".format(x_five_sec[train_index].shape, y_five_sec[train_index].shape))
print("test feature shape & label shape = {0} & {1}".format(x_five_sec[test_index].shape,y_five_sec[test_index].shape))

train feature shape & label shape = (135820, 40) & (135820,)
test feature shape & label shape = (33954, 40) & (33954,)


In [38]:
"""poly_kernel_svm_model = poly_kernel_svm_clf.fit(np.ascontiguousarray(x_five_sec[train_index]) \
                                                , np.ascontiguousarray(y_five_sec[train_index]))"""

poly_kernel_svm_model = poly_kernel_svm_clf.fit(x_train, y_train)

#### this is why its not working as subsetting reduces the labels to 21 classes only

In [31]:
print(np.unique(y_five_sec[train_index]))
print(np.unique(y_five_sec))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63 

In [39]:
poly_kernel_svm_model.classes_

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [40]:
y_predict = poly_kernel_svm_model.predict_proba(x_test)

In [ ]:
y_predict_validation = poly_kernel_svm_model.predict_proba(x_five_sec[test_index])

In [41]:
y_predict.shape

(61119, 182)

In [43]:
sample_labels = [i for i in range(0, 182, 1)]
roc_auc_score(y_test, y_predict, multi_class='ovr', average='macro', labels=sample_labels)

0.9797076605714096

In [44]:
roc_auc_score(y_test, y_predict, multi_class='ovo', average='macro', labels=sample_labels)

0.974440820022206

In [ ]:
roc_auc_score(y_five_sec[test_index], y_predict_validation, multi_class='ovr', average='macro', labels=sample_labels)

In [9]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
features_reduced = pca.fit_transform(pickled_extracted_features_five_sec)

In [11]:
features_reduced.shape

(169774, 2)

In [17]:
df = pd.DataFrame(features_reduced, columns=['e1','e2'])
df['labels'] = labels_five_sec
df["labels"] = df["labels"].astype(str)
df.shape

(169774, 3)

In [18]:
pca_fig = px.scatter(df, x='e1', y='e2', color='labels')
pca_fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [ ]:
# svc_deg3poly_model = load('/kaggle/input/svc-deg3-poly-clf-model/audio_classifier_svc_deg3poly_model.joblib')

In [12]:
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

n_splits = 5
cv = StratifiedKFold(n_splits=n_splits)

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5, probability=True, class_weight="balanced"))
])

# poly_kernel_svm_clf.fit(x_train, y_train)

In [28]:
sample_labels = [i for i in range(0, 182, 1)]

for fold, (train, test) in enumerate(skf.split(x_train, y_train)):
    print("fold = {0}".format(fold))
    x_train = np.ascontiguousarray(x_train[train])
    y_train = np.ascontiguousarray(y_train[train])
    x_test = np.ascontiguousarray(x_train[test])
    y_test = np.ascontiguousarray(y_train[test])
    print("train feature shape & label shape = {0} & {1}".format(x_train.shape, y_train.shape))
    print("test feature shape & label shape = {0} & {1}".format(x_test.shape, y_test.shape))
    poly_kernel_svm_model = poly_kernel_svm_clf.fit(x_train, y_train)
    y_predict = poly_kernel_svm_model.predict_proba(x_test)
    print("predictions shape = {0}".format(y_predict.shape))
    ras_ovr = roc_auc_score(y_test, y_predict, multi_class='ovr', average='macro', labels=sample_labels)
    ras_ovo = roc_auc_score(y_test, y_predict, multi_class='ovo', average='macro', labels=sample_labels)
    print(f"Macro-averaged One-vs-Rest ROC AUC score: ", round(ras_ovr,2))
    print(f"Macro-averaged One-vs-One ROC AUC score: ", round(ras_ovo,2))

fold = 0
train feature shape & label shape = (101864, 40) & (101864,)
test feature shape & label shape = (25466, 40) & (25466,)
predictions shape = (25466, 182)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [47]:
x_train.shape

(135819, 40)

In [37]:
roc_auc_score(y_train[test], y_predict, multi_class='ovr', average='macro', labels=sample_labels)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [29]:
bird_cols = annotated_data['label'].unique().tolist()
df = pd.DataFrame(y_predict, columns=bird_cols)

In [ ]:
solution_sums = df.sum(axis=0)
scored_columns = list(solution_sums[solution_sums > 0].index.values)
assert len(scored_columns) > 0

In [ ]:
df[scored_columns].values.shape

In [ ]:
roc_auc_score(y[test], df[scored_columns].values, multi_class='ovr', average='macro', labels=sample_labels)

In [ ]:
from sklearn.metrics import RocCurveDisplay, auc, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

fpr_grid = np.linspace(0.0, 1.0, 1000)
sample_labels = [i for i in range(0, 182, 1)]

# Interpolate all ROC curves at these points
mean_tpr = np.zeros_like(fpr_grid)

n_classes = len(class_list)
y_onehot_tests, y_scores = dict(), dict()

for fold, (train, test) in enumerate(skf.split(x, y)):
    fpr, tpr, roc_auc, = dict(), dict(), dict()
    print("fold = {0}".format(fold))
    y_score = poly_kernel_svm_clf.fit(x[train], y[train]).predict_proba(x[test])
    y_scores[fold] = y_score
    label_binarizer = LabelBinarizer().fit(y[train])
    y_onehot_test = label_binarizer.transform(y[test])
    y_onehot_tests[fold] = y_onehot_test
    #print("y_onehot_test shape = {0}".format(y_onehot_test.shape))
    
    ras = roc_auc_score(y[test], y_score, multi_class='ovr', average='macro', labels=sample_labels)
    
    """for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_onehot_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    for i in range(n_classes):
        mean_tpr += np.interp(fpr_grid, fpr[i], tpr[i])  # linear interpolation

    # Average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = fpr_grid
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])"""

    #print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc['macro']:.2f}")
    print(f"Macro-averaged One-vs-Rest ROC AUC score: ", round(ras,2))
 

In [ ]:
#class_id = range(n_classes)[91]
#print(class_id)
y_onehot_test = y_onehot_tests[1] ## 0 is the fold number, there are 5 folds
y_score = y_scores[1]
#print(annotated_data['label'].iloc[class_id])

In [ ]:
import matplotlib.colors as mcolors

#len(mcolors.CSS4_COLORS)
color_keys = list(mcolors.CSS4_COLORS.keys())
colors = []

for i in range(0,18):
    #print(color_keys[i])
    color_key = color_keys[i]
    #print(mcolors.CSS4_COLORS[color_key])
    colors.append(mcolors.CSS4_COLORS[color_key])
    
colors[8:18]

In [ ]:
#from itertools import cycle

for i in range(0,190,10):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_facecolor('xkcd:grey')
    bird_list = annotated_data['label'].unique().tolist()
    #colors = cycle(bird_list)

    for class_id, color in zip(range(i,i+10), colors[8:18]):
        if class_id < 182:
            RocCurveDisplay.from_predictions(
                y_onehot_test[:, class_id],
                y_score[:, class_id],
                name=f"ROC curve for {bird_list[class_id]}",
                color=color,
                ax=ax,
            )

        _ = ax.set(
            xlabel="False Positive Rate",
            ylabel="True Positive Rate",
            title="Macro-averaged One-vs-Rest\nReceiver Operating Characteristic for fold 1",
        )

In [ ]:
ros = RandomOverSampler(random_state=42)
features_resampled, labels_resampled = ros.fit_resample(x, y)

print(features_resampled.shape)
print(labels_resampled.shape)

In [ ]:
unique_labels = labels_resampled.unique().sort()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model = random_forest_classifier.fit(x_train, y_train)
y_predict = random_forest_model.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

In [ ]:
y_predict_proba = random_forest_model.predict_proba(x_test)

In [ ]:
roc_score = roc_auc_score(y_test, y_predict_proba, multi_class='ovo', average='macro', labels=unique_labels)
print("ROC AUC Score:", roc_score)

In [ ]:
def evaluate_model(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    # Calculate precision
    precision = precision_score(y_true, y_pred, average='weighted')
    # Calculate recall
    recall = recall_score(y_true, y_pred, average='weighted')
    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1

# Evaluate the model
accuracy, precision, recall, f1 = evaluate_model(y_test, y_predict)
# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
dump(random_forest_model, 'audio_classifier_model.joblib')

In [ ]:
def audio_classification(file_path):
    audio = file_path
    print(audio)
    extracted_features = extract_features(audio).reshape(1, -1)
    y_predict = random_forest_model.predict(extracted_features)
    labels_list = annotated_data['label'].unique()
    encoded_label = annotated_data['encoded_label'].unique()

    labels = {}
    for label, prediction in zip(encoded_label, labels_list):
        labels[label] = prediction
    if y_predict[0] in labels.keys():
        predicted = ('Predicted Class:', labels[y_predict[0]])
    return predicted

In [ ]:
extracted_features = extract_features('/kaggle/input/birdclef-2024/unlabeled_soundscapes/1002849488.ogg').reshape(1, -1)
y_predict_proba = random_forest_model.predict_proba(extracted_features)
y_predict = random_forest_model.predict(extracted_features)

In [ ]:
labels_list = annotated_data['label'].unique()
encoded_label = annotated_data['encoded_label'].unique()

In [ ]:
labels = {}
for label, prediction in zip(encoded_label, labels_list):
    labels[label] = prediction
    
labels.keys()

In [ ]:
labels[y_predict[0]]

In [ ]:
file_path = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1002849488.ogg'
audio_analysis(file_path)
Audio(file_path)

In [ ]:
audio_classification(file_path)

In [ ]:
sample_submission = pl.read_csv('/kaggle/input/birdclef-2024/sample_submission.csv', low_memory=True)
print(sample_submission.shape)
sample_submission.head(10)

In [ ]:
sample_submission2 = sample_submission.drop("row_id")
sample_submission2.sum_horizontal()

In [ ]:
print(np.array(sample_submission2)[2].shape)
print(np.array(sample_submission2).shape)

In [ ]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True, as_frame=False)
print(X.shape)
print(type(y))
print(X.shape)
print(y.shape)
print(X[2])
print(y[2])
print(y[[10, 25, 50]])

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver="liblinear").fit(X, y)
print(clf.predict_proba(X).shape)
print(clf.predict_proba(X)[4].shape)

In [ ]:
sample_labels = [i for i in range(0, 182, 1)]
len(sample_labels)

In [ ]:
y_true_sample = np.array([0,1,2])
roc_auc_score(y_true_sample, sample_submission2, multi_class='ovo', average='macro', labels=sample_labels)

In [ ]:
roc_auc_score(y_true_sample, sample_submission2, multi_class='ovr', average='micro', labels=sample_labels)